In [ ]:
!pip install plotly_express
!wget -O tracelib.py https://raw.githubusercontent.com/HiEST/bojos/master/traces/tracelib.py
#!wget -O emissions_eq_pyc.pyc https://raw.githubusercontent.com/HiEST/bojos/master/traces/emissions_eq_pyc.pyc
!wget -O emissions_eq.py https://raw.githubusercontent.com/HiEST/bojos/master/traces/emissions_eq.py

In [ ]:
import datetime
#from emissions_eq_pyc import *
from emissions_eq import *
from google.colab import files
from tracelib import *

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
traces = pd.read_csv(filename)

In [ ]:
traces['nox'] = traces.apply(lambda row: row.noxme + row.noxae, axis=1)
traces['sox'] = traces.apply(lambda row: row.soxme + row.soxae, axis=1)
traces['co2'] = traces.apply(lambda row: row.co2me + row.co2ae, axis=1)
traces['co'] = traces.apply(lambda row: row.come + row.coae, axis=1)

Cast ts a datetime (tipo de datos consciente del tiempo)

In [ ]:
traces['ts'] = pd.to_datetime(traces['ts'])

# Acumulado de cada traza

In [ ]:
traces.drop(columns = ['uid', 'gid', 'lat', 'lon']).groupby('sid').sum()

# Traza 1

In [ ]:
trace1 = traces[traces.sid == 1]

In [ ]:
pollutant = "co2" # nox, sox, co2, co
plot_trace(trace1, height = 600, pollutant = pollutant)
plot_trace_line(trace1, height = 600)

# Speed + Acceleration

In [ ]:
speed = calculateSpeedAccPandas(trace1)

In [ ]:
plot_line(speed, 'id', 'speed', title="Speed")
plot_line(speed, 'id', 'acc', title="Acceleration")
plot_line(speed, 'id', 'timeDiff', title="Time between messages")

# Emissions

In [ ]:
ship = getShipProfile()

In [ ]:
ship['type'] # Try changing this value to "Yatch" and take a look to the auxiliary engine!

In [ ]:
emis = computeEmissions(ship, speed['speed'], trace1['ts'])

In [ ]:
emis.drop(columns = ['id']).sum()


In [ ]:
plot_line(emis, 'id', 'soxme', title='SOx ME')
plot_line(emis, 'id', 'soxae', title='SOx AE')
plot_line(emis, 'id', 'noxme', title='NOx ME')
plot_line(emis, 'id', 'noxae', title='NOx AE')
plot_line(emis, 'id', 'co2me', title='CO2 ME')
plot_line(emis, 'id', 'co2ae', title='CO2 AE')
plot_line(emis, 'id', 'come', title='CO ME')


In [ ]:
emis

# Ahora os toca a vosotros :)

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
msToKnots = np.float64(1.943844)

##  Speed

In [ ]:
# Ind
def myCalculateSpeed(coords, prevCoords, timeDiff, speedMultiplier=2):
    # Assume that 1º = 111,139 meters
    # Multiply the obtained speed by the speedMultiplier (2)
    # Returns speed in knots <---- KNOTS, OJO!
    return("The earth is flat")

In [ ]:
def myCalculateAcc(timeDiff, speed):
    acc = np.zeros(len(speed))
    # Some calculations here :)
    return(acc)

In [ ]:
def myCalculateSpeedAccPandas(df):
    speed = np.zeros(len(df))
    timeDiff = np.zeros(len(df))
    for i in range(1, len(df)):
        timeDiff[i] = (df.iloc[i]['ts'] - df.iloc[i-1]['ts']).total_seconds()
        speed[i] = calculateSpeed(df.iloc[i], df.iloc[i-1], timeDiff[i])
    acc = calculateAcc(timeDiff, speed)
    return(pd.DataFrame({'id':list(range(0,len(df))),'speed': speed, 'acc': acc, 'timeDiff': timeDiff}))

## Emisiones

In [ ]:
# Individual samples
def myTransientPowerME(VTransient, VDesign, PInstalled, EpsilonP=0.8,
                     VSafety=0.5):
    return(0)

In [ ]:
# Ind
def myTransientPowerAE(speed, shipType, instPow):
    return(0)

In [ ]:
def myCalcSOxEmissionFactor(SFOC=200, SC=0.001):
    return(0)

In [ ]:
def myCalcCO2EmissionFactor(SFOC=200, CC=0.85):
    return(0)

In [ ]:
def myCalcNOxEmissionFactor(rpm):
    return(0)

In [ ]:
def myCalcABC(speedDiff, timeDiff, alpha=582):
    # speed is in knots, we need it in m/s here!
    # np.abs gives us the absolute value of a number
    return(33)

In [ ]:
def myCalcCOEmissionFactor(speedDiff, timeDiff, COBase = 0.8):
    return(1337)

In [ ]:
def myComputeEmissions(ship, speed, time):
    # Static for every step
    NOxEF = calcNOxEmissionFactor(ship['rpm'])
    SOxEF = calcSOxEmissionFactor()
    CO2EF = calcCO2EmissionFactor()

    # Dinamic regarding speed! (Vector!)
    # Duplicate first element
    s = pd.concat([pd.Series(speed.iloc[0]), speed])
    t = pd.concat([pd.Series(time.iloc[0]), time])


    speedDiff = np.diff(s, n=1)
    timeDiff =  np.diff(t, n=1)
    timeDiff = [ max(1, int(t.total_seconds())) for t in timeDiff ]
    
    #COEF = calcCOEmissionFactor(speed, prevSpeed, timeDiff)
    COEF = np.array([ calcCOEmissionFactor(s, t) for s,t in zip(speedDiff, timeDiff) ])

    pME = np.array([ transientPowerME(s, ship['designSpeed'], ship['installedME']) for s in speed ])
    pAE = np.array([ transientPowerAE(s, ship['type'], ship['installedAE']) for s in speed ])

    # Dividing by 3600 makes the emissions from hours to seconds
    emis = pd.DataFrame({
        'id': list(range(0,len(speed))),
        'speed': speed,
        'noxme': (NOxEF * pME / 3600),
        'soxme': (SOxEF * pME / 3600),
        'co2me': (CO2EF * pME / 3600),
        'come': (COEF * pME / 3600),
        'noxae': (NOxEF * pAE / 3600),
        'soxae': (SOxEF * pAE / 3600),
        'co2ae': (CO2EF * pAE / 3600),
        'coae': 0, # (COEF * pAE / 3600),
        'pme': pME,
        'pae': pAE
    })
    return(emis)

In [ ]:
def myGetShipProfile():
    ship = dict({
        'type': 'Passenger/Ro-Ro Cargo Ship',
        'designSpeed': 21.4,
        'installedME': 18006,
        'installedAE': 3420,
        'rpm': 500
    })
    return(ship)

## Pruebas de velocidad

In [ ]:
speed = calculateSpeedAccPandas(trace1)

In [ ]:
myspeed = myCalculateSpeedAccPandas(trace1)

## Pruebas de emisiones

In [ ]:
emis = computeEmissions(ship, speed['speed'], trace1['ts'])
myemis = myComputeEmissions(ship, myspeed['speed'], trace1['ts'])